In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import torchvision.models as models
from torch.utils.data import DataLoader
from torch.optim.lr_scheduler import StepLR
from torch.cuda.amp import autocast, GradScaler
from tqdm import tqdm
import os

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Data Augmentation (optimized for speed)
train_transforms = transforms.Compose([
    transforms.RandomResizedCrop(224),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

test_transforms = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

# Dataset
train_dataset = datasets.Food101(root="data", split="train", download=True, transform=train_transforms)
test_dataset = datasets.Food101(root="data", split="test", download=True, transform=test_transforms)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True, num_workers=2, pin_memory=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False, num_workers=2, pin_memory=True)
print(f"Train size: {len(train_dataset)}, Test size: {len(test_dataset)}")

# Get class names from dataset
class_names = train_dataset.classes
print(f"Total classes: {len(class_names)}")

# Define model structure to match the Streamlit app's FoodClassifier class
class FoodClassifier(nn.Module):
    def __init__(self, num_classes=101):
        super(FoodClassifier, self).__init__()
        # Load pre-trained ResNet50
        self.resnet = models.resnet50(weights=models.ResNet50_Weights.IMAGENET1K_V2)
        # Replace the last fully connected layer
        in_features = self.resnet.fc.in_features
        self.resnet.fc = nn.Sequential(
            nn.Dropout(0.5),
            nn.Linear(in_features, num_classes)
        )

    def forward(self, x):
        return self.resnet(x)

# Create the model
model = FoodClassifier(num_classes=len(class_names))
model = model.to(device)

# PyTorch 2.0 compile (optional)
if hasattr(torch, "compile"):
    model = torch.compile(model)

optimizer = optim.AdamW(model.parameters(), lr=0.001, weight_decay=1e-4)
scheduler = StepLR(optimizer, step_size=5, gamma=0.1)
criterion = nn.CrossEntropyLoss(label_smoothing=0.1)
scaler = GradScaler()

# Training function
def train(model, loader, optimizer, criterion, scaler, epoch):
    model.train()
    total_loss, correct, total = 0, 0, 0

    for images, labels in tqdm(loader, desc=f"Epoch {epoch+1}"):
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()

        with autocast():
            outputs = model(images)
            loss = criterion(outputs, labels)

        scaler.scale(loss).backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
        scaler.step(optimizer)
        scaler.update()

        total_loss += loss.item()
        correct += outputs.argmax(1).eq(labels).sum().item()
        total += labels.size(0)

    acc = 100.0 * correct / total
    return total_loss / len(loader), acc

# Testing function
def test(model, loader, criterion):
    model.eval()
    total_loss, correct, total = 0, 0, 0

    with torch.no_grad():
        for images, labels in loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)
            total_loss += loss.item()
            correct += outputs.argmax(1).eq(labels).sum().item()
            total += labels.size(0)

    acc = 100.0 * correct / total
    return total_loss / len(loader), acc

# Training Loop
num_epochs = 40
best_acc = 0.0
os.makedirs("checkpoints", exist_ok=True)

for epoch in range(num_epochs):
    train_loss, train_acc = train(model, train_loader, optimizer, criterion, scaler, epoch)
    test_loss, test_acc = test(model, test_loader, criterion)
    scheduler.step()

    print(f"Epoch {epoch+1}: Train Acc: {train_acc:.2f}%, Test Acc: {test_acc:.2f}%")

    if test_acc > best_acc:
        best_acc = test_acc
        # Save model in the format expected by the Streamlit app
        torch.save({
            'model_state_dict': model.state_dict(),
            'class_names': class_names,
            'accuracy': best_acc,
            'epoch': epoch,
        }, "checkpoints/best_model.pth")
        print(f"✅ New best model saved: {best_acc:.2f}%")

        # Save the model also with the exact name expected by the Streamlit app
        torch.save({
            'model_state_dict': model.state_dict(),
            'class_names': class_names,
            'accuracy': best_acc,
            'epoch': epoch,
        }, "food101_model_for_inference (1).pth")
        print(f"✅ Model saved for inference with Streamlit app")

# Save the final model
torch.save({
    'model_state_dict': model.state_dict(),
    'class_names': class_names,
    'accuracy': test_acc,
    'epoch': num_epochs,
}, "checkpoints/last_model.pth")

# Also save the final model with the expected name for Streamlit app
torch.save({
    'model_state_dict': model.state_dict(),
    'class_names': class_names,
    'accuracy': test_acc,
    'epoch': num_epochs,
}, "food101_model_for_inference (1).pth")

print(f"🏁 Training Done! Best Test Accuracy: {best_acc:.2f}%")
print(f"Model saved for Streamlit app as 'food101_model_for_inference (1).pth'")

Using device: cuda


100%|██████████| 5.00G/5.00G [01:05<00:00, 75.9MB/s]


Train size: 75750, Test size: 25250
Total classes: 101


Downloading: "https://download.pytorch.org/models/resnet50-11ad3fa6.pth" to /root/.cache/torch/hub/checkpoints/resnet50-11ad3fa6.pth
100%|██████████| 97.8M/97.8M [00:01<00:00, 95.9MB/s]
<ipython-input-1-33cef0e635cc>:70: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = GradScaler()
Epoch 1:   0%|          | 0/1184 [00:00<?, ?it/s]<ipython-input-1-33cef0e635cc>:81: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast():
W0414 12:15:53.222000 206 torch/_inductor/utils.py:1137] [0/0] Not enough SMs to use max_autotune_gemm mode
Epoch 1:   0%|          | 1/1184 [01:47<35:24:44, 107.76s/it]<ipython-input-1-33cef0e635cc>:81: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast():
Epoch 1: 100%|██████████| 1184/1184 [11:27<00:00,  1.72i

Epoch 1: Train Acc: 49.69%, Test Acc: 74.09%
✅ New best model saved: 74.09%
✅ Model saved for inference with Streamlit app


...

Epoch 24: 100%|██████████| 1184/1184 [06:41<00:00,  2.95it/s]


Epoch 24: Train Acc: 82.09%, Test Acc: 87.83%


Epoch 25: 100%|██████████| 1184/1184 [06:42<00:00,  2.94it/s]


Epoch 25: Train Acc: 81.93%, Test Acc: 87.86%


Epoch 26: 100%|██████████| 1184/1184 [06:46<00:00,  2.91it/s]
